<a href="https://colab.research.google.com/github/shishiradk/kindle-review-sentiment-analysis/blob/main/kindle_review_sentimennt_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip uninstall -y numpy
!pip install numpy==1.24.3


Found existing installation: numpy 1.24.3
Uninstalling numpy-1.24.3:
  Successfully uninstalled numpy-1.24.3
  Using cached numpy-1.24.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.6 kB)
Using cached numpy-1.24.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.24.3 which is incompatible.
jax 0.5.2 requires numpy>=1.25, but you have numpy 1.24.3 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.24.3 which is incompatible.
blosc2 3.3.2 requires numpy>=1.26, but you have numpy 1.24.3 which is incompatible.
treescope 0.1.9 requires numpy>=1.25.2, but you have numpy 1.24.3 which is in

In [ ]:
###Best Pratices
# 1.Preprocessing & Cleaning
# 2.Train Test Split
# 3.BOW,TFIDF,Word2vec
# 4.Train ML ALGORITHM

In [ ]:
!pip install gensim

In [ ]:
#load the dataset
import pandas as pd
data = pd.read_csv('all_kindle_review.csv')
data.head()

,Unnamed: 0.1,Unnamed: 0,asin,helpful,rating,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,11539,B0033UV8HI,"[8, 10]",3,"Jace Rankin may be short, but he's nothing to ...","09 2, 2010",A3HHXRELK8BHQG,Ridley,Entertaining But Average,1283385600
1,1,5957,B002HJV4DE,"[1, 1]",5,Great short read. I didn't want to put it dow...,"10 8, 2013",A2RGNZ0TRF578I,Holly Butler,Terrific menage scenes!,1381190400
2,2,9146,B002ZG96I4,"[0, 0]",3,I'll start by saying this is the first of four...,"04 11, 2014",A3S0H2HV6U1I7F,Merissa,Snapdragon Alley,1397174400
3,3,7038,B002QHWOEU,"[1, 3]",3,Aggie is Angela Lansbury who carries pocketboo...,"07 5, 2014",AC4OQW3GZ919J,Cleargrace,very light murder cozy,1404518400
4,4,1776,B001A06VJ8,"[0, 1]",4,I did not expect this type of book to be in li...,"12 31, 2012",A3C9V987IQHOQD,Rjostler,Book,1356912000


In [ ]:
df=data[['reviewText','rating']]
df.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",3
1,Great short read. I didn't want to put it dow...,5
2,I'll start by saying this is the first of four...,3
3,Aggie is Angela Lansbury who carries pocketboo...,3
4,I did not expect this type of book to be in li...,4


In [ ]:
df.shape

(12000, 2)

In [ ]:
##missing values
df.isnull().sum()

,0
reviewText,0
rating,0


In [ ]:
df['rating'].unique()

array([3, 5, 4, 2, 1])

In [ ]:
df['rating'].value_counts()

,count
rating,
5,3000
4,3000
3,2000
2,2000
1,2000


In [ ]:
##Preprocessing and cleaning

#positive review is 1 and negative is 0
df['rating']=df['rating'].apply(lambda x:0 if x<3 else 1)

<ipython-input-8-16ab188acfc3>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['rating']=df['rating'].apply(lambda x:0 if x<3 else 1)


In [ ]:
df['rating'].value_counts()

,count
rating,
1,8000
0,4000


In [ ]:
## 1. Lower all the cases
df['reviewText']=df['reviewText'].str.lower()

<ipython-input-10-1c21b4e0537c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['reviewText']=df['reviewText'].str.lower()


In [ ]:
df.head()

,reviewText,rating
0,"jace rankin may be short, but he's nothing to ...",1
1,great short read. i didn't want to put it dow...,1
2,i'll start by saying this is the first of four...,1
3,aggie is angela lansbury who carries pocketboo...,1
4,i did not expect this type of book to be in li...,1


In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from bs4 import BeautifulSoup

In [ ]:

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# Step 1: Remove special characters
df.loc[:, 'reviewText'] = df['reviewText'].apply(lambda x: re.sub(r'[^a-zA-Z0-9\s]', '', str(x)))

# Step 2: Remove stopwords
stop_words = set(stopwords.words('english'))
df.loc[:, 'reviewText'] = df['reviewText'].apply(lambda x: " ".join([y for y in str(x).split() if y.lower() not in stop_words]))

# Step 3: Remove URLs
df.loc[:, 'reviewText'] = df['reviewText'].apply(lambda x: re.sub(r'https?://\S+|www\.\S+', '', str(x)))

# Step 4: Remove HTML tags
df.loc[:, 'reviewText'] = df['reviewText'].apply(lambda x: BeautifulSoup(x, 'lxml').get_text() if isinstance(x, str) else '')

# Step 5: Normalize whitespace
df.loc[:, 'reviewText'] = df['reviewText'].apply(lambda x: " ".join(str(x).split()))

In [ ]:
df.head()

,reviewText,rating
0,jace rankin may short hes nothing mess man hau...,1
1,great short read didnt want put read one sitti...,1
2,ill start saying first four books wasnt expect...,1
3,aggie angela lansbury carries pocketbooks inst...,1
4,expect type book library pleased find price right,1


In [ ]:
##lemmatizer
from nltk.stem import WordNetLemmatizer

In [ ]:
lemmatizer=WordNetLemmatizer()

In [ ]:
def lemmatize_words(text):
  return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

In [ ]:
 nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
df.loc[:, 'reviewText'] = df['reviewText'].apply(lambda x:lemmatize_words(x))

In [ ]:
df.head()

,reviewText,rating
0,jace rankin may short he nothing mess man haul...,1
1,great short read didnt want put read one sitti...,1
2,ill start saying first four book wasnt expecti...,1
3,aggie angela lansbury carry pocketbook instead...,1
4,expect type book library pleased find price right,1


In [ ]:
##train test split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(df['reviewText'],df['rating'],test_size=0.20)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
bow=CountVectorizer()
X_train_bow=bow.fit_transform(X_train).toarray()
X_test_bow=bow.transform(X_test).toarray()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer()
# Use fit_transform on the training data to learn the vocabulary and idf weights
X_train_tfidf=tfidf.fit_transform(X_train).toarray()
# Use transform on the test data with the learned vocabulary and idf weights
X_test_tfidf=tfidf.transform(X_test).toarray()

In [ ]:
X_train_bow

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
from sklearn.naive_bayes import GaussianNB
nb_model_bow=GaussianNB().fit(X_train_bow,y_train)
nb_model_tfidf=GaussianNB().fit(X_train_tfidf,y_train)

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report

In [ ]:
y_pred_bow = nb_model_bow.predict(X_test_bow)


In [ ]:
y_pred_tfidf = nb_model_tfidf.predict(X_test_tfidf)


In [ ]:
confusion_matrix(y_test,y_pred_bow)

array([[498, 282],
       [748, 872]])

In [ ]:
print("BOW accuracy :",accuracy_score(y_test,y_pred_bow))

BOW accuracy : 0.5708333333333333


In [ ]:
print("TFIDF accuracy:",accuracy_score(y_test,y_pred_tfidf))

TFIDF accuracy: 0.5695833333333333


In [ ]:
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import nltk

In [ ]:
from nltk.tokenize import word_tokenize

In [ ]:
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
X_train_tokens = [word_tokenize(text) for text in X_train]
X_test_tokens = [word_tokenize(text) for text in X_test]

In [ ]:
w2v_model = Word2Vec(sentences=X_train_tokens, vector_size=100, window=5, min_count=1, workers=4)

In [ ]:
def get_avg_word2vec(tokens, model, vector_size):
    vec = np.zeros(vector_size)
    count = 0
    for word in tokens:
        if word in model.wv:
            vec += model.wv[word]
            count += 1
    if count != 0:
        vec /= count
    return vec

In [ ]:
import numpy as np

vector_size = 100

X_train_w2v = np.array([get_avg_word2vec(tokens, w2v_model, vector_size) for tokens in X_train_tokens])
X_test_w2v = np.array([get_avg_word2vec(tokens, w2v_model, vector_size) for tokens in X_test_tokens])


In [ ]:

nb_model_w2v = GaussianNB().fit(X_train_w2v, y_train)
y_pred_w2v = nb_model_w2v.predict(X_test_w2v)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

print("Word2Vec accuracy:", accuracy_score(y_test, y_pred_w2v))
print(confusion_matrix(y_test, y_pred_w2v))
print(classification_report(y_test, y_pred_w2v))

Word2Vec accuracy: 0.6779166666666666
[[ 572  208]
 [ 565 1055]]
              precision    recall  f1-score   support

           0       0.50      0.73      0.60       780
           1       0.84      0.65      0.73      1620

    accuracy                           0.68      2400
   macro avg       0.67      0.69      0.66      2400
weighted avg       0.73      0.68      0.69      2400

